In [28]:
# Import initial dependencies
import pandas as pd
import numpy as np

In [29]:
# Convert CSVs to DFs
features = "../output/UPDATED_reduced_wine_features.csv"
targets = "../output/reduced_wine_targets.csv"

features_df = pd.read_csv(features)
targets_df =pd.read_csv(targets) 

In [30]:
# Check feature DF head
features_df.head()

,white,price,country_Austria,"flavor_categories_taste_notes_light, fruity"
0,0.0,15.0,0,1
1,0.0,65.0,0,0
2,0.0,19.0,0,1
3,0.0,34.0,0,0
4,0.0,30.0,0,0


In [31]:
# Check feature DF shape
features_df.shape

(109662, 4)

In [32]:
# Check targets DF
targets_df.head()

,points
0,87
1,87
2,87
3,87
4,87


In [33]:
# Check Dtypes
targets_df.dtypes

points    int64
dtype: object

In [34]:
# Check feature and target DF shape
print(features_df.shape, targets_df.shape)

(109662, 4) (109662, 1)


In [35]:
# Assign the features and target to X and y abd check assignments
raw_feature_data = features_df.values
raw_target_data = targets_df.values
X = raw_feature_data[:, 0:5]
y = raw_target_data.reshape(-1,1)

print(X, y)

[[ 0. 15.  0.  1.]
 [ 0. 65.  0.  0.]
 [ 0. 19.  0.  1.]
 ...
 [ 0. 70.  0.  0.]
 [ 0. 25.  0.  0.]
 [ 0. 38.  0.  0.]] [[87]
 [87]
 [87]
 ...
 [90]
 [90]
 [91]]


# SPLIT, NORMALIZE, AND ENCODE THE DATA

In [36]:
# Create the train and test sets for the features and target
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [37]:
# Scale the features using the MinMax scaler since we know their values
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_scaler = MinMaxScaler().fit(y_train)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

# HYPERPERAMETER TUNING

In [38]:
# Import the sequential and dense modules fo build my NN
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm

# Run XGB Regressor with unscalled data and recommended parameters from the previous test

In [39]:
print('Select Model...')
from xgboost import XGBRegressor
from datetime import datetime
import pprint
import os

# Create start time and model for XGB Regressor
start_time  = datetime.now()
xgb_reg = XGBRegressor() 


parameters_reg = {'n_estimators': [850], 'max_depth':[5, 7], \
              'learning_rate':[.01], 'batch_size':[10, 16]}

grid_search = GridSearchCV(estimator=xgb_reg, param_grid=parameters_reg, cv=10, n_jobs=-1)


print("parameters:")
pprint.pprint(parameters_reg)


grid = grid_search.fit(X_train_scaled, y_train_scaled.ravel())


print("Best score: %0.3f" % grid.best_score_)
print("Best parameters set:")
best_parameters=grid.best_estimator_.get_params()


for param_name in sorted(best_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    end_time = datetime.now()
print(f'Select Done..., Time Cost: {((end_time - start_time).seconds)}.') 

Select Model...
parameters:
{'batch_size': [10, 16],
 'learning_rate': [0.01],
 'max_depth': [5, 7],
 'n_estimators': [850]}
[13:18:29] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best score: 0.401
Best parameters set:
	base_score: 0.5
	batch_size: 10
	booster: 'gbtree'
	colsample_bylevel: 1
	colsample_bynode: 1
	colsample_bytree: 1
	gamma: 0
	importance_type: 'gain'
	learning_rate: 0.01
	max_delta_step: 0
	max_depth: 5
	min_child_weight: 1
	missing: None
	n_estimators: 850
	n_jobs: 1
	nthread: None
	objective: 'reg:linear'
	random_state: 0
	reg_alpha: 0
	reg_lambda: 1
	scale_pos_weight: 1
	seed: None
	silent: None
	subsample: 1
	verbosity: 1
Select Done..., Time Cost: 200.


In [40]:
from sklearn.metrics import mean_squared_error, r2_score

# Use our model to predict a value
predicted = grid.predict(X_test_scaled)

# Score the prediction with mse and r2
mse = mean_squared_error(y_test_scaled, predicted)
r2 = r2_score(y_test_scaled, predicted)

# Print scores
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

Mean Squared Error (MSE): 0.01410573966214706
R-squared (R2): 0.3979035733416677


In [41]:
# Create list of predictions and check
predicted_list = grid.predict(X_test_scaled)

predicted_list

array([0.43168384, 0.29862463, 0.365847  , ..., 0.40303826, 0.5965271 ,
       0.3683988 ], dtype=float32)

In [42]:
# Create list of actual values and check
actual_list = y_test_scaled

actual_list

array([[0.55],
       [0.45],
       [0.45],
       ...,
       [0.55],
       [0.8 ],
       [0.5 ]])

In [43]:
# Create DF for unscaled test to further compare predictions with actual points
prediction_comparrisons_df = pd.DataFrame({'XGB_Predictions (unscaled)' : predicted_list,
                                           'Actual_Points (unscaled)' : actual_list.ravel()
                                          })

prediction_comparrisons_df.head(40)

,XGB_Predictions (unscaled),Actual_Points (unscaled)
0,0.431684,0.55
1,0.298625,0.45
2,0.365847,0.45
3,0.410079,0.40
4,0.563481,0.40
5,0.383288,0.65
6,0.481746,0.40
7,0.543273,0.60
8,0.391415,0.25
9,0.449901,0.35


In [44]:
# Export unscaled results
prediction_comparrisons_df.to_csv(r'C:\Users\howar\Desktop\Data Science Boot Camp\Group_Project_3\prediction_modeling\output\UPDATED_predictions_vs_actual.csv', index=False)

In [45]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\XGB_Regressor_unscaled_(BEST_SCALED_REDUCED_FEATURES).sav'
joblib.dump(grid, filename)

['saved_models\\XGB_Regressor_unscaled_(BEST_SCALED_REDUCED_FEATURES).sav']

In [50]:
print('Select Model...')
from xgboost import XGBRegressor
from datetime import datetime
import pprint
import os


xgb_reg_2 = XGBRegressor() 


parameters_reg_2 = {'n_estimators': [850], 'max_depth':[5], \
              'learning_rate':[.01], 'batch_size':[10]}

grid_search_2 = GridSearchCV(estimator=xgb_reg_2, param_grid=parameters_reg_2, cv=10, n_jobs=-1)


print("parameters:")
pprint.pprint(parameters_reg_2)


grid_2 = grid_search_2.fit(X_train, y_train.ravel())


print("Best score: %0.3f" % grid_2.best_score_)
print("Best parameters set:")
best_parameters_2=grid_2.best_estimator_.get_params()


for param_name in sorted(parameters_reg_2.keys()):
    print("\t%s: %r" % (param_name, best_parameters_2[param_name]))



Select Model...
parameters:
{'batch_size': [10],
 'learning_rate': [0.01],
 'max_depth': [5],
 'n_estimators': [850]}
[13:23:07] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best score: 0.401
Best parameters set:
	batch_size: 10
	learning_rate: 0.01
	max_depth: 5
	n_estimators: 850


In [51]:
from sklearn.metrics import mean_squared_error, r2_score

# Use our model to predict a value
predicted = grid_2.predict(X_test)

# Score the prediction with mse and r2
mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)

# Print scores
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

Mean Squared Error (MSE): 5.643206485438499
R-squared (R2): 0.3978063998842317


In [52]:
# Create list of predictions and check
predicted_list_unscaled = grid_2.predict(X_test)

predicted_list_unscaled

array([88.645454, 85.942245, 87.31623 , ..., 88.03888 , 91.90177 ,
       87.33857 ], dtype=float32)

In [53]:
# Create list of actual values and check
actual_list_unscaled = y_test

actual_list_unscaled

array([[91],
       [89],
       [89],
       ...,
       [91],
       [96],
       [90]], dtype=int64)

In [54]:
# Create DF for unscaled test to further compare predictions with actual points
prediction_comparrisons_unscaled_df = pd.DataFrame({'XGB_Predictions (unscaled)' : predicted_list_unscaled,
                                           'Actual_Points (unscaled)' : actual_list_unscaled.ravel()
                                          })

prediction_comparrisons_unscaled_df.head(40)

,XGB_Predictions (unscaled),Actual_Points (unscaled)
0,88.645454,91
1,85.942245,89
2,87.316231,89
3,88.237907,88
4,91.251907,88
5,87.658455,93
6,89.626060,88
7,90.849518,92
8,87.821381,85
9,88.979530,87


In [55]:
# Export unscaled results
prediction_comparrisons_unscaled_df.to_csv(r'C:\Users\howar\Desktop\Data Science Boot Camp\Group_Project_3\prediction_modeling\output\UPDATED_predictions_vs_actual_unscaled.csv', index=False)

In [56]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\XGB_Regressor_unscaled_(BEST_UNSCALED_REDUCED_FEATURES).sav'
joblib.dump(grid_2, filename)

['saved_models\\XGB_Regressor_unscaled_(BEST_UNSCALED_REDUCED_FEATURES).sav']

In [60]:
# Create list of predictions and check
predicted_list_full = grid_2.predict(X)

predicted_list_full

array([86.126945, 90.67158 , 87.01504 , ..., 91.08157 , 88.09046 ,
       89.15921 ], dtype=float32)

In [61]:
predicted_list_full.shape

(109662,)

In [58]:
# Create list of actual values and check
actual_list_full = y

actual_list_full

array([[87],
       [87],
       [87],
       ...,
       [90],
       [90],
       [91]], dtype=int64)

In [59]:
actual_list_full.shape

(109662, 1)

In [64]:
features_df['predicted_scores'] = predicted_list_full

In [65]:
features_df.head()

,white,price,country_Austria,"flavor_categories_taste_notes_light, fruity",predicted_scores
0,0.0,15.0,0,1,86.126945
1,0.0,65.0,0,0,90.671577
2,0.0,19.0,0,1,87.015038
3,0.0,34.0,0,0,88.775177
4,0.0,30.0,0,0,88.502922


In [68]:
features_df.shape

(109662, 5)

In [66]:
# Export unscaled results
features_df.to_csv(r'C:\Users\howar\Desktop\Data Science Boot Camp\Group_Project_3\prediction_modeling\output\full_predictions.csv', index=False)

In [67]:
features_df['price'].nunique()

382

# UPDATE TO FEATURE SET FOR EASE OF HANDLING IN THE APP...

In [ ]:
# Convert CSVs to DFs
UPDATED_features = "UPDATED_reduced_wine_features.csv"

UPDATED_features_df = pd.read_csv(UPDATED_features)

UPDATED_features_df.head(2)

# SEE WHAT A CLASSIFIER DOES WITH THE DATA

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Test other models for comparrison
# NOTE, THESE ARE CLASSIFICATION MODELS I AM TRYIN OUT JUST TO SEE HOW IT PERFORMS ON THE DATA
params_clf={'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
            'normalize': [True, False]}


# Creat, fit, and test model
rdg_clf = Ridge()
clf_grid = GridSearchCV(rdg_clf, params_clf, cv=2, verbose = 1, scoring = 'neg_mean_squared_error')
clf_grid.fit(X_train_scaled, y_train_scaled)

In [ ]:
# Print the score and best params for the activation model
print("Best: %f using %s" % (clf_grid.best_score_, clf_grid.best_params_))

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Use our model to predict a value
predicted_3 = clf_grid.predict(X_test_scaled)

# Score the prediction with mse and r2
mse_3 = mean_squared_error(y_test_scaled, predicted_3)
r2_3 = r2_score(y_test_scaled, predicted_3)

# Print results
print(f"Mean Squared Error (MSE): {mse_3}")
print(f"R-squared (R2): {r2_3}")

In [ ]:
# Create list of predictions and check
predicted_list_clf = clf_grid.predict(X_test)

predicted_list_clf

In [ ]:
# Create list of actual values and check
actual_list_clf = y_test

actual_list_clf

In [ ]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\Ridge_Classifier.sav'
joblib.dump(clf_grid, filename)

# TESTING OUT OTHER MODEL EXAMPLES I FOUND

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score

# Try some different model examples I found when doing my research...
def cross_validate_best_known():
    '''
        import and clean the wine data, then do a corss validation on each of the three models we are
        training here. A RandomForest, a GradientBoost, and an AdaBoost backed by a DecisionTree. Print
        the scores.

        The parameters we're using here are the "best" that we've found so far using a grid search.
    '''

rf = RandomForestRegressor(max_features=2, min_samples_split=4, n_estimators=50, min_samples_leaf=2)
gb = GradientBoostingRegressor(loss='quantile', learning_rate=0.0001, n_estimators=50, max_features='log2', min_samples_split=2, max_depth=1)
ada_tree_backing = DecisionTreeRegressor(max_features='sqrt', splitter='random', min_samples_split=4, max_depth=3)
ab = AdaBoostRegressor(ada_tree_backing, learning_rate=0.0001, loss='square', n_estimators=100000)

# Fit the data to the Random Forest Regressor
rf.fit(X_train_scaled, y_train_scaled.ravel())


# validate.cross_v_scores([rf, gb, ab], X_train_scaled, y_train_scaled)
# # RandomForestRegressor -- RMLSE: -0.596797712098, R2: 0.0272065373946
# GradientBoostingRegressor -- RMLSE: -0.996134592541, R2: -2.37202164829
# AdaBoostRegressor -- RMLSE: -0.706385708459, R2: -0.103966980393 

In [ ]:
# Use our model to predict a value
predicted_rf = rf.predict(X_test_scaled)

# Score the prediction with mse and r2
mse_rf = mean_squared_error(y_test_scaled, predicted_rf)
r2_rf = r2_score(y_test_scaled, predicted_rf)

# Print results
print(f"Mean Squared Error (MSE): {mse_rf}")
print(f"R-squared (R2): {r2_rf}")

In [ ]:
# Create list of predictions and check
predicted_list_rf = rf.predict(X_test_scaled)

predicted_list_rf

In [ ]:
# Create list of actual values and check
actual_list_rf = y_test_scaled

actual_list_rf

In [ ]:
# Create list of predictions and check
predicted_list_unscaled_rf = rf.predict(X_test)

predicted_list_unscaled_rf

In [ ]:
# Create list of actual values and check
actual_list_unscaled_rf = y_test

actual_list_unscaled_rf

In [ ]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\RF_Regr.sav'
joblib.dump(rf, filename)

In [ ]:
# Fit the data to the Gradient Boosting Regressor
gb.fit(X_train_scaled, y_train_scaled.ravel())

In [ ]:
# Use our model to predict a value
predicted_4 = gb.predict(X_test_scaled)

# Score the prediction with mse and r2
mse_4 = mean_squared_error(y_test_scaled, predicted_4)
r2_4 = r2_score(y_test_scaled, predicted_4)

# Print results
print(f"Mean Squared Error (MSE): {mse_4}")
print(f"R-squared (R2): {r2_4}")

In [ ]:
# Create list of predictions and check
predicted_list_gb = gb.predict(X_test_scaled)

predicted_list_gb

In [ ]:
# Create list of actual values and check
actual_list_gb = y_test_scaled

actual_list_gb

In [ ]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\GB_Regr.sav'
joblib.dump(gb, filename)

In [ ]:
# Fit the data to the ADA Tree Backing Regressor
ada_tree_backing.fit(X_train_scaled, y_train_scaled.ravel())

In [ ]:
# Use our model to predict a value
predicted_5 = ada_tree_backing.predict(X_test_scaled)

# Score the prediction with mse and r2
mse_5 = mean_squared_error(y_test_scaled, predicted_5)
r2_5 = r2_score(y_test_scaled, predicted_5)

# Print Results
print(f"Mean Squared Error (MSE): {mse_5}")
print(f"R-squared (R2): {r2_5}")

In [ ]:
# Create list of predictions and check
predicted_list_ada_tree_backing = ada_tree_backing.predict(X_test_scaled)

predicted_list_ada_tree_backing

In [ ]:
# Create list of actual values and check
actual_list_ada_tree_backing = y_test_scaled

actual_list_ada_tree_backing

In [ ]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\ADA_Tree_Backing.sav'
joblib.dump(ada_tree_backing, filename)

In [ ]:
# Fit the data to the ADA Boosting Regressor
ab.fit(X_train_scaled, y_train_scaled.ravel())

In [ ]:
# Use our model to predict a value
predicted_6 = ab.predict(X_test_scaled)

# Score the prediction with mse and r2
mse_6 = mean_squared_error(y_test_scaled, predicted_6)
r2_6 = r2_score(y_test_scaled, predicted_6)

# Print Results
print(f"Mean Squared Error (MSE): {mse_6}")
print(f"R-squared (R2): {r2_6}")

In [ ]:
# Create list of predictions and check
predicted_list_ab = ab.predict(X_test_scaled)

predicted_list_ab

In [ ]:
# Create list of actual values and check
actual_list_ab = y_test_scaled

actual_list_ab

In [ ]:
# Create list of predictions and check
predicted_list_ab = ab.predict(X_test)

predicted_list_ab

In [ ]:
# Create list of actual values and check
actual_list_ab = y_test

actual_list_ab

In [ ]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\AB_Reg.sav'
joblib.dump(ab, filename)

# TESTING MY BUILD OF THE KERAS REGRESSOR

In [ ]:
# Create the base Keras classifier wrapper for activation method
def create_base_model(activation='relu'):
    
    model = Sequential()
    model.add(Dense(units=1000, activation='relu', input_dim=11))
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dense(units=1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model


# Create activation model
keras_model = KerasRegressor(build_fn=create_base_model, epochs=20, batch_size=88)


# Define the activation grid search parameters
batch_size = [10, 32, 64, 88, 100, 150]


# Assign the activation grid search parameters, fit, and run the model
keras_param_grid = dict(batch_size=batch_size)
keras_grid = GridSearchCV(estimator=keras_model, param_grid=keras_param_grid, n_jobs=-1)
keras_grid_result = keras_grid.fit(X_train_scaled, y_train_scaled)

In [ ]:
# Print the score and best params for the activation model
print("Best: %f using %s" % (keras_grid_result.best_score_, activation_grid_result.best_params_))

In [ ]:
print(f"Training Data Score: {keras_grid_result.score(X_train_scaled, y_train_scaled)}")
print(f"Testing Data Score: {keras_grid_result.score(X_train_scaled, y_train_scaled)}")

In [ ]:
# Use our model to predict a value
predicted_7 = keras_grid_result.predict(X_test_scaled)

# Score the prediction with mse and r2
mse_7 = mean_squared_error(y_test_scaled, predicted_7)
r2_7 = r2_score(y_test_scaled, predicted_7)

# Print Results
print(f"Mean Squared Error (MSE): {mse_7}")
print(f"R-squared (R2): {r2_7}")

In [ ]:
# Create list of predictions and check
predicted_list_keras = keras_grid_result.predict(X_test_scaled)

predicted_list_keras

In [ ]:
# Create list of actual values and check
actual_list_keras = y_test_scaled

actual_list_keras

In [ ]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\Keras_Reg_NN.sav'
joblib.dump(keras_grid_result, filename)

In [ ]:
# # Create the base Keras classifier wrapper for the model
# def create_model(activation='adam', dropout_rate=0.0):
    
#     model = Sequential()
#     model.add(Dense(units=100, activation='relu', input_dim=5))
#     model.add(Dense(units=100, activation='relu'))
#     model.add(Dense(units=100, activation='tanh'))
#     model.add(Dense(units=100, activation='relu'))
#     model.add(Dense(units=3, activation='softmax'))
#     model.add(Dropout(dropout_rate))
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
#     return model


# # Create model
# model = KerasClassifier(build_fn=create_model, epochs=250, batch_size=88)


# # Define the activation and dropout rate grid search parameters
# activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


# # Assign the grid search parameters, fit, and run the model
# param_grid = dict(activation=activation, dropout_rate=dropout_rate)
# fit_grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
# grid_result = fit_grid.fit(X_train_scaled, y_train_categorical)

In [ ]:
# # Print the score and best params for the activation model
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
# # Create the base Keras classifier wrapper for optimizer model
# def create_optimizer_model(optimizer='adam'):
    
#     model = Sequential()
#     model.add(Dense(units=100, activation='hard_sigmoid', input_dim=5))
#     model.add(Dense(units=100, activation='hard_sigmoid'))
#     model.add(Dense(units=100, activation='hard_sigmoid'))
#     model.add(Dense(units=100, activation='hard_sigmoid'))
#     model.add(Dense(units=3, activation='hard_sigmoid'))
#     model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
#     return model


# # Create optimizer model
# optimizer_model = KerasClassifier(build_fn=create_optimizer_model, epochs=250, batch_size=88)


# # Define the optimizer grid search parameters
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']


# # Assign the optimizer grid search parameters, fit, and run the model
# optimizer_param_grid = dict(optimizer=optimizer)
# optimizer_grid = GridSearchCV(estimator=optimizer_model, param_grid=optimizer_param_grid, n_jobs=-1)
# optimizer_grid_result = optimizer_grid.fit(X_train_scaled, y_train_categorical)

In [ ]:
# # Print the score and best params for the optimizer model
# print("Best: %f using %s" % (optimizer_grid_result.best_score_, optimizer_grid_result.best_params_))

In [ ]:
# # Create the base Keras classifier wrapper for dropout model
# def create_dropout_model(dropout_rate=0.0):
    
#     model = Sequential()
#     model.add(Dense(units=100, activation='relu', input_dim=5))
#     model.add(Dense(units=100, activation='relu'))
#     model.add(Dense(units=100, activation='tanh'))
#     model.add(Dense(units=100, activation='relu'))
#     model.add(Dense(units=3, activation='softmax'))
#     model.add(Dropout(dropout_rate))
#     model.compile(loss='categorical_crossentropy', optimizer='Adadelta', metrics=['accuracy']) 
    
#     return model


# # Create the dropout model
# dropout_model = KerasClassifier(build_fn=create_dropout_model, epochs=250, batch_size=88)


# # Define the dropout grid search parameters
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


# # Assign the dropout grid search parameters, fit, and run the model
# dropout_param_grid = dict(dropout_rate=dropout_rate)
# dropout_grid = GridSearchCV(estimator=dropout_model, param_grid=dropout_param_grid, n_jobs=-1)
# dropout_grid_result = dropout_grid.fit(X_train_scaled, y_train_categorical)

In [ ]:
# # Print the score and best params for the dropout model
# print("Best: %f using %s" % (dropout_grid_result.best_score_, dropout_grid_result.best_params_))

In [ ]:
# print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
# print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")